In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

### Yahoo S5

In [5]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [6]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(128, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        latent_dim = 16

        encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
        x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
        x = layers.Conv1D(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Flatten()(x)
        x = layers.Dense(128, activation="relu")(x)
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
        
        latent_inputs = keras.Input(shape=(latent_dim,))
        x = layers.Dense(32* 64, activation="relu")(latent_inputs)
        x = layers.Reshape((32, 64))(x)
        x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Flatten()(x)
        x = layers.Dense(X_train.shape[1] * X_train.shape[2])(x)
        decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
        decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
        
        model = VAE(encoder, decoder)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
        history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

yahoo_A1 0.5714285224489823 0.16666666570436459 0.416666641666668
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.666666577777783 0.2499999625000044 0.8888887901234668
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 0.0
yahoo_A1 0.46153841893491365 0.14999999350000026 0.49999997619047704
yahoo_A1 0.6666666133333363 0.4214285474078342 0.3999999830000005
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.8333332708333363 0.7930951820434847 0.8799999648000009
yahoo_A1 0.3636363272727291 0.11111110432098803 0.49999996875000174
yahoo_A1 0.9090908429752097 0.7223809201395328 0.7249999496666695
yahoo_A1 0.6666666133333363 0.2494444395492283 0.17999999230000024
yahoo_A1 0.666666577777783 0.6249998958333479 0.9687499269531303
yahoo_A1 0.4999999375000028 0.16666664444444684 0.7777776913580334
yahoo_A1 0.399999960000002 0.07142857308673395 0.14285713367346983
yahoo_A1 0.18181816198347142 0.049999994500000554 0.49999994444445006
yahoo_A1 0.7142856581632683 0.47992060771546535 0.5499999772500006
ya

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

yahoo_A3 0.5333332871111137 0.10000000693578515 0.49999996770833555
yahoo_A3 0.5882352456747433 0.1820761156452809 0.10714285293367355
yahoo_A3 0.736842050969532 1.4367633554077069e-08 0.0
yahoo_A3 0.4999999562500026 0.1041666691166871 0.12499999427083351
yahoo_A3 0.8749999398437528 0.620833296813961 0.5833333028769854
yahoo_A3 0.4999999562500026 0.140211637158566 0.1249999921875004
yahoo_A3 0.749999943750003 0.42043649389383175 0.43333331522222274
yahoo_A3 0.7058822989619408 0.18134920550654918 0.36111109444444517
yahoo_A3 0.7777777216049413 0.4601983939112567 0.44374997831250085
yahoo_A3 0.0 0.0 -0.0
yahoo_A3 0.9565216809073751 0.7412783168094822 0.12499998593750142
yahoo_A3 0.9565216809073751 0.0 0.0
yahoo_A3 0.799999944000003 0.3761309429913805 0.1499999932500002
yahoo_A3 0.3636363272727291 0.0625000016493049 0.29999998200000094
yahoo_A3 0.736842050969532 0.22812499803616132 -0.0
yahoo_A3 0.16666664861111138 0.045454540495868265 0.09090908099173658
yahoo_A3 0.9473683612188393 0.628

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

yahoo_A4 0.6666666148148177 0.40849204766817154 0.527777757546297
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.4999999562500026 0.0714285775077294 0.24999997187500275
yahoo_A4 0.6666666148148177 0.12321428488378036 0.3333333202777782
yahoo_A4 0.4999999562500026 0.07142857955137984 0.2499999843750008
yahoo_A4 0.999999930000003 0.9166665921675762 0.999999930000004
yahoo_A4 0.736842050969532 0.3207804158246333 0.1999999920000002
yahoo_A4 0.5333332871111137 0.16736110679852345 0.24999998750000055
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.5333332871111137 2.7651513124356705e-09 0.0
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.9090908512396723 0.861892142833962 0.7152777334587216
yahoo_A4 0.736842050969532 0.2596230115339825 0.04999999775000007
yahoo_A4 0.9090908512396723 0.4724702285105039 -0.0
yahoo_A4 0.6666666148148177 0.1269841325481336 0.0833333277777781
yahoo_A4 0.736842050969532 0.08333334512160165 0.0
yahoo_A4 0.0 0.0 -0.0
yahoo_A4 0.3999999626666689 0.0 0.0
yahoo_A4 0.15384613727810673 0.04166666215277823 -0.0
yah

In [7]:
yahoo_results = pd.DataFrame(total_scores)

In [8]:
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.425643,0.208009,0.270768
yahoo_A2,0.000000,0.000000,0.000000
yahoo_A3,0.656001,0.276384,0.237310
yahoo_A4,0.556930,0.191848,0.156115


### NASA

In [9]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [10]:
for loader in [load_nasa]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        latent_dim = 16

        encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
        x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
        x = layers.Conv1D(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Flatten()(x)
        x = layers.Dense(128, activation="relu")(x)
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
        
        latent_inputs = keras.Input(shape=(latent_dim,))
        x = layers.Dense(32* 64, activation="relu")(latent_inputs)
        x = layers.Reshape((32, 64))(x)
        x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Flatten()(x)
        x = layers.Dense(X_train.shape[1] * X_train.shape[2])(x)
        decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
        decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
        
        model = VAE(encoder, decoder)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
        history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.25388598800874246 0.1414733094195465 0.5005733789658734
D2 0.2699530282282525 0.15229549424478592 0.5155843225011233


In [11]:
nasa_results = pd.DataFrame(total_scores)

In [12]:
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.253886,0.141473,0.500573
D2,0.269953,0.152295,0.515584


### SMD

In [13]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [ ]:
for loader in [load_smd]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        latent_dim = 16

        encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
        x = layers.Conv1D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
        x = layers.Conv1D(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Flatten()(x)
        x = layers.Dense(128, activation="relu")(x)
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
        
        latent_inputs = keras.Input(shape=(latent_dim,))
        x = layers.Dense(32* 64, activation="relu")(latent_inputs)
        x = layers.Reshape((32, 64))(x)
        x = layers.Conv1DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
        x = layers.Flatten()(x)
        x = layers.Dense(X_train.shape[1] * X_train.shape[2])(x)
        decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
        decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
        
        model = VAE(encoder, decoder)
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
        history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.21943571700356884 0.06161971811210573 0.49999999837062536
smd 0.13733904288530016 0.028173112021507282 0.36261987746582136
smd 0.19999995422223202 0.14219012503143807 0.5787897130193578
smd 0.18604648513911787 0.11866806227026533 0.5133701159873187
smd 0.33333328333333917 0.35640820541890905 0.6211790312653149
smd 0.4532373739169843 0.2669330983626245 0.5179672349984987
smd 0.2831049957173562 0.14461924966945794 0.5173868870979054
smd 0.20895519513255004 0.11940347522366578 0.547204295148935
smd 0.22033895027291484 0.1666149344974412 0.5691633546205771
smd 0.28363633908892766 0.3241394928242778 0.5605232313902969
smd 0.1044176607022476 0.03525980884669687 0.4900122252606943
smd 0.2710622474312033 0.09125372910904292 0.23458540739482808
smd 0.22813686174442485 0.146714004750293 0.4791606156205909
smd 0.15384613727810673 0.2113461199160637 0.5001515106639715
smd 0.2580644795005248 0.2009804413350092 0.46037670393170643
smd 0.7999999200000041 0.6687762990145876 0.8333333051994312
sm

In [ ]:
smd_results = pd.DataFrame(total_scores)

In [ ]:
smd_results.groupby('dataset').mean()